In [1]:
import numpy as np
import neonutilities as nu
import pandas as pd 
from neonutilities.aop_download import validate_dpid,validate_site_format,validate_neon_site
from neonutilities.helper_mods.api_helpers import get_api
import pickle 
import geopandas as gpd

## Custom function to list available dates for a given data product and site

In [12]:
def nu_list_available_dates(dpid, site):
    """
		NOTE: This is an internal hack of the original function to return a dataframe instead of printing
        
        nu_list_available_dates displays the available releases and dates for a given product and site
        --------
         Inputs:
             dpid: the data product code (eg. 'DP3.30015.001' - CHM)
             site: the 4-digit NEON site code (eg. 'JORN')
        --------
        Returns:
        prints the Release Tag (or PROVISIONAL) and the corresponding available dates (YYYY-MM) for each tag
    --------
        Usage:
        --------
        >>> list_available_dates('DP3.30015.001','JORN')
        RELEASE-2025 Available Dates: 2017-08, 2018-08, 2019-08, 2021-08, 2022-09

        >>> list_available_dates('DP3.30015.001','HOPB')
        PROVISIONAL Available Dates: 2024-09
        RELEASE-2025 Available Dates: 2016-08, 2017-08, 2019-08, 2022-08

        >>> list_available_dates('DP1.10098.001','HOPB')
        ValueError: There are no data available for the data product DP1.10098.001 at the site HOPB.
    """
    product_url = "https://data.neonscience.org/api/v0/products/" + dpid
    response = get_api(api_url=product_url)  # add input for token?

    # raise value error and print message if dpid isn't formatted as expected
    validate_dpid(dpid)

    # raise value error and print message if site is not a 4-letter character
    site = site.upper()  # make site upper case (if it's not already)
    validate_site_format(site)

    # raise value error and print message if site is not a valid NEON site
    validate_neon_site(site)

    # check if product is active
    if response.json()["data"]["productStatus"] != "ACTIVE":
        raise ValueError(
            f"NEON {dpid} is not an active data product. See https://data.neonscience.org/data-products/{dpid} for more details."
        )

    # get available releases & months:
    for i in range(len(response.json()["data"]["siteCodes"])):
        if site in response.json()["data"]["siteCodes"][i]["siteCode"]:
            available_releases = response.json()["data"]["siteCodes"][i][
                "availableReleases"
            ]

    # display available release tags (including provisional) and dates for each tag
    try:
        availables_list = []
        for entry in available_releases:
            release = entry["release"]
            available_months_str = ", ".join(entry["availableMonths"])
            available_months = [x.strip() for x in available_months_str.split(',')]
            for available_month in available_months:
                availables_list.append({'status':release,'date':available_month})
        available_df = pd.DataFrame(availables_list)
        return(available_df)
    except UnboundLocalError:
        # if the available_releases variable doesn't exist, this error will show up:
        # UnboundLocalError: local variable 'available_releases' referenced before assignment
        raise ValueError(
            f"There are no NEON data available for the data product {dpid} at the site {site}."
        )

In [3]:
site_names = ['DELA','LENO','TALL','BONA','DEJU','HEAL','SRER','SJER','SOAP',
              'TEAK','CPER','NIWO','RMNP','DSNY','OSBS','JERC','PUUM','KONZ',
              'UKFS','SERC','HARV','UNDE','BART','JORN','DCFS','NOGP','WOOD',
              'GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS',
              'SCBI','ABBY','WREF','STEI','TREE','YELL']

In [ ]:
import time 

vst_path = '/data/chloris/NEON/DP1.10098/'

out_path = '/ssdS/data/NEON/'

dp_id = "DP1.30003.001"  #point clouds 

# Read the plot polygon location file 
allplots_gdf = gpd.read_file('s3://chloris-data-us-west-2/projects/NEON/NEON_TOS_Plot_Polygons.geojson')


site_worked = []
site_failed = []

# Loop the sites
for site_name in site_names[1:]:

	# Just the plots for the site in question 
	siteplots_gdf = allplots_gdf[allplots_gdf['siteID']==site_name]

	# Get the available years
	obs_years = nu_list_available_dates(dpid = dp_id, site=site_name)
	obs_years['year'] = obs_years['date'].str.slice(0,4).astype(int)

	# Loop all years 
	for obs_year in obs_years['year'].sort_values():

		start_timer = time.time()
		try:
			nu.by_tile_aop(dpid=dp_id, site=site_name, year=obs_year, 
					easting=list(siteplots_gdf.easting), 
					northing=list(siteplots_gdf.northing),
					include_provisional=True,
					check_size =False,
					savepath=out_path)
			
			print(f"Downloaded {site_name} {obs_year} in {time.time() - start_timer} seconds")
			site_worked.append(f"{site_name}_{obs_year}")

		except Exception as e:
			print(f"Failed for site {site_name} {obs_year}: {e}")
			site_failed.append(f"{site_name}_{obs_year}")

Provisional NEON data are included. To exclude provisional data, use input parameter include_provisional=False.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:35<00:00,  2.37it/s]


Downloaded LENO 2015 in 40.35420799255371 seconds


Provisional NEON data are included. To exclude provisional data, use input parameter include_provisional=False.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:40<00:00,  2.08it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 211/211 [07:29<00:00,  2.13s/it]


Downloaded YELL 2023 in 454.5527741909027 seconds


Provisional NEON data are included. To exclude provisional data, use input parameter include_provisional=False.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [02:29<00:00,  4.15s/it]


Downloaded YELL 2025 in 153.00627326965332 seconds


In [19]:
site_worked

['LENO_2015',
 'LENO_2016',
 'LENO_2017',
 'LENO_2018',
 'LENO_2019',
 'LENO_2021',
 'LENO_2023',
 'LENO_2024',
 'TALL_2015',
 'TALL_2016',
 'TALL_2017',
 'TALL_2018',
 'TALL_2019',
 'TALL_2021',
 'TALL_2023',
 'TALL_2024',
 'BONA_2017',
 'BONA_2018',
 'BONA_2019',
 'BONA_2021',
 'BONA_2023',
 'BONA_2024',
 'DEJU_2017',
 'DEJU_2018',
 'DEJU_2019',
 'DEJU_2021',
 'DEJU_2023',
 'DEJU_2024',
 'HEAL_2017',
 'HEAL_2018',
 'HEAL_2019',
 'HEAL_2021',
 'HEAL_2023',
 'HEAL_2024',
 'SRER_2017',
 'SRER_2018',
 'SRER_2019',
 'SRER_2021',
 'SRER_2022',
 'SRER_2024',
 'SJER_2013',
 'SJER_2017',
 'SJER_2018',
 'SJER_2019',
 'SJER_2021',
 'SJER_2023',
 'SJER_2024',
 'SOAP_2013',
 'SOAP_2017',
 'SOAP_2018',
 'SOAP_2019',
 'SOAP_2021',
 'SOAP_2023',
 'SOAP_2023',
 'SOAP_2024',
 'TEAK_2013',
 'TEAK_2017',
 'TEAK_2018',
 'TEAK_2019',
 'TEAK_2021',
 'TEAK_2023',
 'TEAK_2024',
 'CPER_2013',
 'CPER_2017',
 'CPER_2020',
 'CPER_2020',
 'CPER_2021',
 'CPER_2024',
 'CPER_2024',
 'NIWO_2017',
 'NIWO_2018',
 'NIWO

In [ ]:
site_names = ['DELA','LENO','TALL','BONA','DEJU','HEAL','SRER','SJER','SOAP',
              'TEAK','CPER','NIWO','RMNP','DSNY','OSBS','JERC','PUUM','KONZ',
              'UKFS','SERC','HARV','UNDE','BART','JORN','DCFS','NOGP','WOOD',
              'GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS',
              'SCBI','ABBY','WREF','STEI','TREE','YELL']

site_name = site_names[0]

# Read the full vegetation structure dict for this site 
with open(f"{vst_path.rstrip('/')}/{site_name}.pkl", "rb") as f:
    vst_dict = pickle.load(f)

# Get the apparent individual dataframe
apparent_df = vst_dict['vst_apparentindividual']

# Get unique individual IDs
individual_ids = np.unique(apparent_df['individualID'])

# Get the site plot ids
plot_ids = np.unique(apparent_df['namedLocation'])

# For each individual, find the corresponding plot

apparent_df = vst_dict['vst_apparentindividual']
apparent_df